# Joining Data in SQL

> the power of joining tables while exploring interesting features of countries and their cities throughout the world

- author: Victor Omondi
- toc: true
- comments: true
- categories: [joins, sql]
- image: images/jds-shield-png

# Overview

We'll explore the power of joining tables while exploring interesting features of countries and their cities throughout the world. We will use inner and outer joins, as well as self joins, semi joins, anti joins and cross joins—fundamental tools in any PostgreSQL wizard's toolbox.

# Setup 

In [1]:
import pandas as pd
%load_ext sql
%sql sqlite://

'Connected: @None'

# Introduction to joins

We'll be exploring the concept of joining tables, and will explore the different ways to enrich queries using inner joins and self joins. We'll also see how to use the case statement to split up a field into different categories.

## Introduction to INNER JOIN

### prime_ministers table

In [2]:
prime_ministers = pd.read_csv("datasets/leaders/prime_ministers.csv")
prime_ministers.head()

,country,continent,prime_minister
0,Egypt,Africa,Sherif Ismail
1,Portugal,Europe,Antonio Costa
2,Vietnam,Asia,Nguyen Xuan Phuc
3,Haiti,North America,Jack Guy Lafontant
4,India,Asia,Narendra Modi


In [3]:
presidents = pd.read_csv("datasets/leaders/presidents.csv")
presidents.head()

,country,continent,president
0,Egypt,Africa,Abdel Fattah el-Sisi
1,Portugal,Europe,Marcelo Rebelo de Sousa
2,Haiti,North America,Jovenel Moise
3,Uruguay,South America,Jose Mujica
4,Liberia,Africa,Ellen Johnson Sirleaf


In [4]:
%sql DROP TABLE IF EXISTS prime_ministers;
%sql PERSIST prime_ministers;
%sql DROP TABLE IF EXISTS presidents;
%sql PERSIST presidents;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted presidents'

### INNER JOIN in SQL

In [5]:
%%sql
SELECT p1.country, p1.continent, prime_minister, president
    FROM prime_ministers AS p1
    INNER JOIN presidents AS p2
    ON p1.country = p2.country

 * sqlite://
Done.


country,continent,prime_minister,president
Egypt,Africa,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Europe,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Asia,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,North America,Jack Guy Lafontant,Jovenel Moise


We'll be working with the `countries` database containing information about the most populous world cities as well as country-level economic data, population data, and geographic data. This `countries` database also contains information on languages spoken in each country.

In [6]:
cities = pd.read_csv("datasets/countries/cities.csv")
cities.head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop
0,Abidjan,CIV,4765000,NaN,4765000
1,Abu Dhabi,ARE,1145000,NaN,1145000
2,Abuja,NGA,1235880,6000000.0,1235880
3,Accra,GHA,2070463,4010054.0,2070463
4,Addis Ababa,ETH,3103673,4567857.0,3103673


In [7]:
countries = pd.read_csv("datasets/countries/countries.csv")
countries.head()

,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.17610,34.5228
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,Nederland,Constitutional Monarchy,Amsterdam,4.89095,52.3738
2,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,Shqiperia,Republic,Tirane,19.81720,41.3317
3,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,Al-Jazair/Algerie,Republic,Algiers,3.05097,36.7397
4,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,Amerika Samoa,US Territory,Pago Pago,-170.69100,-14.2846


In [8]:
%sql DROP TABLE IF EXISTS cities;
%sql DROP TABLE IF EXISTS countries;
%sql PERSIST cities;
%sql PERSIST countries;

 * sqlite://
Done.
 * sqlite://
Done.
 * sqlite://
 * sqlite://


'Persisted countries'

In [9]:
%%sql
-- Select all columns from cities
SELECT *
    FROM cities
    LIMIT 5;

 * sqlite://
Done.


index,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop
0,Abidjan,CIV,4765000,None,4765000
1,Abu Dhabi,ARE,1145000,None,1145000
2,Abuja,NGA,1235880,6000000.0,1235880
3,Accra,GHA,2070463,4010054.0,2070463
4,Addis Ababa,ETH,3103673,4567857.0,3103673


In [10]:
%%sql
SELECT * 
FROM cities
    -- 1. Inner join to countries
    INNER JOIN countries
    -- 2. Match on the country codes
    ON cities.country_code = countries.code
    LIMIT 5;

 * sqlite://
Done.


index,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop,index_1,code,country_name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,Abidjan,CIV,4765000,None,4765000,133,CIV,Cote d'Ivoire,Africa,Western Africa,322463.0,1960.0,Cote dIvoire,Republic,Yamoussoukro,-4.0305,5.332000000000001
1,Abu Dhabi,ARE,1145000,None,1145000,8,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.3705,24.4764
2,Abuja,NGA,1235880,6000000.0,1235880,131,NGA,Nigeria,Africa,Western Africa,923768.0,1960.0,Nigeria,Federal Republic,Abuja,7.48906,9.05804
3,Accra,GHA,2070463,4010054.0,2070463,52,GHA,Ghana,Africa,Western Africa,238533.0,1957.0,Ghana,Republic,Accra,-0.20795,5.57045
4,Addis Ababa,ETH,3103673,4567857.0,3103673,45,ETH,Ethiopia,Africa,Eastern Africa,1104300.0,-1000.0,YeItyop´iya,Republic,Addis Ababa,38.7468,9.022739999999999


In [11]:
%%sql
-- 1. Select name fields (with alias) and region 
SELECT cities.name AS city, countries.country_name AS country, countries.region
    FROM cities
    INNER JOIN countries
    ON cities.country_code = countries.code
    LIMIT 5;

 * sqlite://
Done.


city,country,region
Abidjan,Cote d'Ivoire,Western Africa
Abu Dhabi,United Arab Emirates,Middle East
Abuja,Nigeria,Western Africa
Accra,Ghana,Western Africa
Addis Ababa,Ethiopia,Eastern Africa


Instead of writing the full table name, we can use table aliasing as a shortcut. For tables we also use AS to add the alias immediately after the table name with a space. To select a field in the query that appears in multiple tables, we'll need to identify which table/table alias we're referring to by using a `.` in the `SELECT` statement.

We'll now explore a way to get data from both the `countries` and `economies` tables to examine the inflation rate for both 2010 and 2015.

In [12]:
economies = pd.read_csv("datasets/countries/economies.csv")
economies.head()

,econ_id,code,year,income_group,gdp_percapita,gross_savings,inflation_rate,total_investment,unemployment_rate,exports,imports
0,1,AFG,2010,Low income,539.667,37.133,2.179,30.402,NaN,46.394,24.381
1,2,AFG,2015,Low income,615.091,21.466,-1.549,18.602,NaN,-49.110,-7.294
2,3,AGO,2010,Upper middle income,3599.270,23.534,14.480,14.433,NaN,-3.266,-21.076
3,4,AGO,2015,Upper middle income,3876.200,-0.425,10.287,9.552,NaN,6.721,-21.778
4,5,ALB,2010,Upper middle income,4098.130,20.011,3.605,31.305,14.0,10.645,-8.013


In [13]:
%sql DROP TABLE IF EXISTS economies
%sql PERSIST economies;

 * sqlite://
Done.
 * sqlite://


'Persisted economies'

In [14]:
%%sql
-- 3. Select fields with aliases
SELECT c.code AS country_code, country_name, year, inflation_rate
    FROM countries AS c
    -- 1. Join to economies (alias e)
    INNER JOIN economies AS e
    -- 2. Match on code
    ON c.code = e.code
    LIMIT 5;

 * sqlite://
Done.


country_code,country_name,year,inflation_rate
AFG,Afghanistan,2010,2.1790000000000003
AFG,Afghanistan,2015,-1.5490000000000002
NLD,Netherlands,2010,0.932
NLD,Netherlands,2015,0.22
ALB,Albania,2010,3.605


In [15]:
populations = pd.read_csv("datasets/countries/populations.csv")
populations.head()

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,20,ABW,2010,1.704,74.953537,101597.0
1,19,ABW,2015,1.647,75.573585,103889.0
2,2,AFG,2010,5.746,58.970829,27962207.0
3,1,AFG,2015,4.653,60.717171,32526562.0
4,12,AGO,2010,6.416,50.654171,21219954.0


In [16]:
%sql DROP TABLE IF EXISTS populations;
%sql PERSIST populations;

 * sqlite://
Done.
 * sqlite://


'Persisted populations'

Now, for each country, we want to get the country name, its region, and the fertility rate and unemployment rate for both 2010 and 2015.

In [17]:
%%sql
-- 4. Select fields
SELECT code, country_name, region, year, fertility_rate
    -- 1. From countries (alias as c)
    FROM countries as c
    -- 2. Join with populations (as p)
    INNER JOIN populations as p
    -- 3. Match on country code
    ON code = country_code
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate
AFG,Afghanistan,Southern and Central Asia,2010,5.746
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005
NLD,Netherlands,Western Europe,2010,1.79
NLD,Netherlands,Western Europe,2015,1.71
ALB,Albania,Southern Europe,2010,1.663


In [18]:
%%sql
-- 6. Select fields
SELECT c.code, country_name, region, e.year, fertility_rate, unemployment_rate
    -- 1. From countries (alias as c)
    FROM countries AS c
    -- 2. Join to populations (as p)
    INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
    -- 4. Join to economies (as e)
    INNER JOIN economies as e
    -- 5. Match on country code
    ON c.code = e.code
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate,unemployment_rate
AFG,Afghanistan,Southern and Central Asia,2010,4.6530000000000005,None
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005,None
AFG,Afghanistan,Southern and Central Asia,2010,5.746,None
AFG,Afghanistan,Southern and Central Asia,2015,5.746,None
NLD,Netherlands,Western Europe,2010,1.71,4.995


In [19]:
%%sql
-- 6. Select fields
SELECT c.code, country_name, region, e.year, fertility_rate, unemployment_rate
    -- 1. From countries (alias as c)
    FROM countries AS c
    -- 2. Join to populations (as p)
    INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
     -- 4. Join to economies (as e)
    INNER JOIN economies AS e
    -- 5. Match on country code and year
    ON c.code = e.code AND e.year=p.year
    LIMIT 5;

 * sqlite://
Done.


code,country_name,region,year,fertility_rate,unemployment_rate
AFG,Afghanistan,Southern and Central Asia,2010,5.746,None
AFG,Afghanistan,Southern and Central Asia,2015,4.6530000000000005,None
NLD,Netherlands,Western Europe,2010,1.79,4.995
NLD,Netherlands,Western Europe,2015,1.71,6.891
ALB,Albania,Southern Europe,2010,1.663,14.0


## INNER JOIN via USING


`INNER JOIN `requires a specification of the key field (or fields) in each table.

When joining tables with a common field name You can use `USING` as a shortcut

In [20]:
countries.rename(columns={"country_name":"name"}, inplace=True)
%sql DROP TABLE IF EXISTS countries;
%sql PERSIST countries; 

 * sqlite://
Done.
 * sqlite://


'Persisted countries'

In [23]:
languages = pd.read_csv("datasets/countries/languages.csv")
languages.head()

,lang_id,code,name,percent,official
0,1,AFG,Dari,50.0,True
1,2,AFG,Pashto,35.0,True
2,3,AFG,Turkic,11.0,False
3,4,AFG,Other,4.0,False
4,5,ALB,Albanian,98.8,True


In [24]:
%sql DROP TABLE IF EXISTS languages;
%sql PERSIST languages

 * sqlite://
Done.
 * sqlite://


'Persisted languages'

In [25]:
%%sql
-- 4. Select fields
SELECT c.name AS country, continent, l.name AS language, official
  -- 1. From countries (alias as c)
  FROM countries c
  -- 2. Join to languages (as l)
  INNER JOIN languages l
    -- 3. Match using code
    USING(code)
    LIMIT 5;

 * sqlite://
Done.


country,continent,language,official
Afghanistan,Asia,Dari,1
Afghanistan,Asia,Other,0
Afghanistan,Asia,Pashto,1
Afghanistan,Asia,Turkic,0
Netherlands,Europe,Dutch,1


## Self-ish joins, just in CASE


### Join prime_ministers to itself?

In [29]:
%%sql
SELECT p1.country country1, p2.country country2, p1.continent
    FROM prime_ministers p1
    INNER JOIN prime_ministers p2
    ON p1.continent=p2.continent AND p1.country<>p2.country
    LIMIT(5)

 * sqlite://
Done.


country1,country2,continent
Portugal,Norway,Europe
Portugal,Spain,Europe
Vietnam,Brunei,Asia
Vietnam,India,Asia
Vietnam,Oman,Asia


### Creating indep_year_group in states

In [30]:
states = pd.read_csv("datasets/leaders/states.csv")
states.head()

,name,continent,indep_year,fert_rate,women_parli_perc
0,Australia,Oceania,1901,1.88,32.74
1,Brunei,Asia,1984,1.96,6.06
2,Chile,South America,1810,1.80,15.82
3,Egypt,Africa,1922,2.70,14.90
4,Haiti,North America,1804,3.03,2.74


In [31]:
%sql DROP TABLE IF EXISTS states;
%sql PERSIST states;

 * sqlite://
Done.
 * sqlite://


'Persisted states'

In [38]:
%%sql
SELECT name, continent, indep_year,
    CASE WHEN indep_year < 1900 THEN "Before 1900"
        WHEN indep_year <= 1930 THEN "Between 1900 AND 1930"
        ELSE "After 1930"
    END AS indep_year_group
    FROM states
    ORDER BY indep_year_group
    LIMIT 13;

 * sqlite://
Done.


name,continent,indep_year,indep_year_group
Brunei,Asia,1984,After 1930
India,Asia,1947,After 1930
Oman,Asia,1951,After 1930
Vietnam,Asia,1945,After 1930
Chile,South America,1810,Before 1900
Haiti,North America,1804,Before 1900
Liberia,Africa,1847,Before 1900
Portugal,Europe,1143,Before 1900
Spain,Europe,1492,Before 1900
Uruguay,South America,1828,Before 1900


### Self-join

We'll use the `populations` table to perform a self-join to calculate the percentage increase in population from 2010 to 2015 for each country code!

Since we'll be joining the `populations` table to itself, We will alias `populations as p1` and also `populations as p2`. 

> Important: This is good practice whenever we are aliasing and tables have the same first letter.


> Note: We are required to alias the tables with self-joins.

In [40]:
%%sql
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM populations AS p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = p2.year - 5
        LIMIT 5;

 * sqlite://
Done.


country_code,size2010,size2015,growth_perc
ABW,101597.0,103889.0,2.255972125161176
AFG,27962207.0,32526562.0,16.3233002316305
AGO,21219954.0,25021974.0,17.917192468937493
ALB,2913021.0,2889167.0,-0.8188749754979453
AND,84419.0,70473.0,-16.519977730131842


### Case when and then

Often it's useful to look at a numerical field not as raw data, but instead as being in different categories or groups.

We can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new grouping field.

In [54]:
%%sql
CREATE TABLE countries_plus AS
SELECT name, continent, code, surface_area,
    -- 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        -- 2. Second case
        WHEN surface_area > 350000 AND surface_area<=2000000 THEN "medium"
        -- 3. Else clause + end
        ELSE "small" END
        -- 4. Alias name
        AS geosize_group
-- 5. From table
FROM countries;

 * sqlite://
Done.


[]

In [55]:
%%sql
SELECT * 
    FROM countries_plus
    LIMIT 5;

 * sqlite://
Done.


name,continent,code,surface_area,geosize_group
Afghanistan,Asia,AFG,652090.0,medium
Netherlands,Europe,NLD,41526.0,small
Albania,Europe,ALB,28748.0,small
Algeria,Africa,DZA,2381740.0,large
American Samoa,Oceania,ASM,199.0,small


### Inner challenge

We will now explore the relationship between the size of a country in terms of surface area and in terms of population using grouping fields created with CASE.

In [57]:
%%sql
CREATE TABLE pop_plus AS
SELECT country_code, size,
  CASE WHEN size > 50000000
            THEN 'large'
       WHEN size > 1000000
            THEN 'medium'
       ELSE 'small' END
       AS popsize_group       
FROM populations
WHERE year = 2015;

SELECT * FROM pop_plus
    LIMIT 5;

 * sqlite://
Done.
Done.


country_code,size,popsize_group
ABW,103889.0,small
AFG,32526562.0,medium
AGO,25021974.0,medium
ALB,2889167.0,medium
AND,70473.0,small


In [59]:
%%sql
-- 5. Select fields
SELECT name, continent, geosize_group, popsize_group
-- 1. From countries_plus (alias as c)
FROM countries_plus AS c
  -- 2. Join to pop_plus (alias as p)
  INNER JOIN pop_plus AS p
    -- 3. Match on country code
    ON c.code = p.country_code
-- 4. Order the table    
ORDER BY geosize_group
LIMIT 5;

 * sqlite://
Done.


name,continent,geosize_group,popsize_group
Algeria,Africa,large,medium
Argentina,South America,large,medium
Australia,Oceania,large,medium
Brazil,South America,large,large
Greenland,North America,large,small


# Outer joins and cross joins

We'll come to grips with different kinds of outer joins. We'll explore how to gain further insights into our data through left joins, right joins, and full joins. In addition to outer joins, we'll also work with cross joins.



## LEFT and RIGHT JOINs

### The syntax of a LEFT JOIN

In [61]:
%%sql
SELECT p1.country, prime_minister, president
    FROM prime_ministers AS p1
    LEFT JOIN presidents AS p2
    ON p1.country = p2.country;

 * sqlite://
Done.


country,prime_minister,president
Egypt,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,Jack Guy Lafontant,Jovenel Moise
India,Narendra Modi,None
Australia,Malcolm Turnbull,None
Norway,Erna Solberg,None
Brunei,Hassanal Bolkiah,None
Oman,Qaboos bin Said al Said,None
Spain,Mariano Rajoy,None


### Left Join

Now we'll explore the differences between performing an inner join and a left join using the `cities` and `countries` tables.

We'll begin by performing an inner join with the `cities` table on the left and the `countries` table on the right.

We will then change the query to a left join.

In [62]:
%%sql
-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1
  -- Join to right table (with alias)
  INNER JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC
LIMIT 10;

 * sqlite://
Done.


city,code,country,region,city_proper_pop
Harare,ZWE,Zimbabwe,Eastern Africa,1606000
Lusaka,ZMB,Zambia,Eastern Africa,1742979
Cape Town,ZAF,South Africa,Southern Africa,3740026
Durban,ZAF,South Africa,Southern Africa,3442361
Ekurhuleni,ZAF,South Africa,Southern Africa,3178470
Johannesburg,ZAF,South Africa,Southern Africa,4434827
Sana'a,YEM,Yemen,Middle East,1937451
Hanoi,VNM,Vietnam,Southeast Asia,6844100
Ho Chi Minh City,VNM,Vietnam,Southeast Asia,7681700
Caracas,VEN,Venezuela,South America,1943901


In [64]:
%%sql
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  -- 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    -- 2. Match on country code
    ON c1.country_code = c2.code
-- 3. Order by descending country code
ORDER BY code DESC
LIMIT 10;

 * sqlite://
Done.


city,code,country,region,city_proper_pop
Harare,ZWE,Zimbabwe,Eastern Africa,1606000
Lusaka,ZMB,Zambia,Eastern Africa,1742979
Cape Town,ZAF,South Africa,Southern Africa,3740026
Durban,ZAF,South Africa,Southern Africa,3442361
Ekurhuleni,ZAF,South Africa,Southern Africa,3178470
Johannesburg,ZAF,South Africa,Southern Africa,4434827
Sana'a,YEM,Yemen,Middle East,1937451
Hanoi,VNM,Vietnam,Southeast Asia,6844100
Ho Chi Minh City,VNM,Vietnam,Southeast Asia,7681700
Caracas,VEN,Venezuela,South America,1943901


> Note: The `INNER JOIN` version resulted in 230 records. The `LEFT JOIN` version returned 236 rows.

### Left join (2)
Next, we'll try out another example comparing an inner join to its corresponding left join. 

We will begin with an inner join on the `countries` table on the left with the languages table on the right. Then we'll change the code to a left join in the next bullet.

Note the use of multi-line comments here using /* and */

In [65]:
%%sql
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on fields
    USING(code)
-- 4. Order by descending country
ORDER BY country DESC
LIMIT 10;

 * sqlite://
Done.


country,local_name,language,percent
Zimbabwe,Zimbabwe,Chewa,None
Zimbabwe,Zimbabwe,Chibarwe,None
Zimbabwe,Zimbabwe,English,None
Zimbabwe,Zimbabwe,Kalanga,None
Zimbabwe,Zimbabwe,Koisan,None
Zimbabwe,Zimbabwe,Nambya,None
Zimbabwe,Zimbabwe,Ndau,None
Zimbabwe,Zimbabwe,Ndebele,None
Zimbabwe,Zimbabwe,Shangani,None
Zimbabwe,Zimbabwe,Shona,None


In [66]:
%%sql
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  LEFT JOIN languages AS l
    -- 3. Match on fields
    USING(code)
-- 4. Order by descending country
ORDER BY country DESC
LIMIT 10;

 * sqlite://
Done.


country,local_name,language,percent
Zimbabwe,Zimbabwe,Chewa,None
Zimbabwe,Zimbabwe,Chibarwe,None
Zimbabwe,Zimbabwe,English,None
Zimbabwe,Zimbabwe,Kalanga,None
Zimbabwe,Zimbabwe,Koisan,None
Zimbabwe,Zimbabwe,Nambya,None
Zimbabwe,Zimbabwe,Ndau,None
Zimbabwe,Zimbabwe,Ndebele,None
Zimbabwe,Zimbabwe,Shangani,None
Zimbabwe,Zimbabwe,Shona,None


> Note: The `INNER JOIN` version resulted in 909 records. The `LEFT JOIN` version returned 916 rows.

We will use `AVG()` in combination with left join to determine the average gross domestic product (GDP) per capita by region in 2010.

In [67]:
%%sql
-- Select fields
SELECT region, AVG(gdp_percapita) AS avg_gdp
-- From countries (alias as c)
FROM countries c
  -- Left join with economies (alias as e)
  LEFT JOIN economies e
    -- Match on code fields
    USING(code)
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region
-- Order by descending avg_gdp
ORDER BY avg_gdp DESC
LIMIT 10;

 * sqlite://
Done.


region,avg_gdp
Western Europe,58130.962857142855
Nordic Countries,57073.998
North America,47911.509999999995
Australia and New Zealand,44792.384999999995
British Islands,43588.33
Eastern Asia,26205.851666666666
Southern Europe,22926.410909090908
Middle East,18204.64176470588
Baltic Countries,12631.029999999999
Caribbean,11413.339461538462


### Right join
Right joins aren't as common as left joins. One reason why is that you can always write a right join as a left join.

In [68]:
%%sql
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM cities
  LEFT JOIN countries
    ON cities.country_code = countries.code
  LEFT JOIN languages
    ON countries.code = languages.code
ORDER BY city, language
LIMIT 10;

 * sqlite://
Done.


city,urbanarea_pop,country,indep_year,language,percent
Abidjan,4765000,Cote d'Ivoire,1960.0,French,None
Abidjan,4765000,Cote d'Ivoire,1960.0,Other,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Arabic,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,English,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Hindi,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Persian,None
Abu Dhabi,1145000,United Arab Emirates,1971.0,Urdu,None
Abuja,1235880,Nigeria,1960.0,English,None
Abuja,1235880,Nigeria,1960.0,Fulani,None
Abuja,1235880,Nigeria,1960.0,Hausa,None


## FULL JOINs
